## Note this code takes 10 hours to run
Partially because I dont know how to implement semaphore pattern in python
The thing is with multiprocessing, if one process encounters 429 HTTP error caused by  request throtteling, all of the processes need to be put to sleep for 1/0.5 second. If you can do this, please commit to the repo.
All the data is available for downloads in the repo's /data section except for the original dataset which you can find on [the lens dataset frontpage](https://grouplens.org/datasets/movielens/20m/)

In [1]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [4]:
movies = pd.read_csv("data/ml-20m/movies.csv")
links = pd.read_csv("data/ml-20m/links.csv")
ratings = pd.read_csv("data/ml-20m/ratings.csv")
tags = pd.read_csv("data/ml-20m/tags.csv")

# Movies Parsing and Preprocessing

## Tmdb parsing

In [6]:
# ! pip install tmdbsimple
import tmdbsimple as tmdb
import numpy as np
import pickle
import time
tmdb.API_KEY = 'cf3539645a105ad8ab9613b7f587a9f7'
movie_infos = {}

In [ ]:
failed = []
for idx in tqdm(links.dropna().index):
    try:
        movie_infos[links['movieId'][idx]] = tmdb.Movies(links['tmdbId'][idx]).info()
    except:
        failed.append(idx)

In [ ]:
pickle.dump(movie_infos,  open( "data/infos_tmdb.p", "wb" ))
print(failed)

In [5]:
failed = [4113, 4473, 4973, 5112, 6559, 7356, 7410, 7416, 7459, 7516, 7517, 8808, 8851, 8852, 8953, 8968, 8975, 8989, 9038, 9063, 9087, 9107, 9161, 9177, 9181, 9225, 9250, 9295, 9395, 9438, 9447, 9462, 9617, 9623, 9660, 9716, 9890, 10324, 10465, 10474, 10732, 10774, 10791, 11339, 11520, 11633, 11654, 11770, 11885, 11934, 11938, 11955, 12124, 12356, 12513, 12713, 12875, 12963, 13040, 13092, 13093, 13133, 13184, 13318, 13468, 13522, 13693, 13694, 13927, 13936, 13996, 14045, 14113, 14151, 14185, 14590, 14681, 14688, 14837, 14878, 14900, 15129, 15163, 15164, 15175, 15208, 15229, 15255, 15289, 15550, 15681, 15771, 15890, 15923, 16164, 16744, 16784, 16946, 17018, 17077, 17248, 17305, 17492, 17564, 17565, 17933, 18072, 18101, 18175, 18351, 18469, 18515, 18577, 18610, 18691, 18698, 18894, 18903, 18970, 18982, 19016, 19093, 19205, 19233, 19234, 19250, 19254, 19332, 19337, 19401, 19427, 19428, 19431, 19442, 19443, 19494, 19677, 19942, 19993, 20040, 20161, 20226, 20295, 20296, 20373, 20374, 20380, 20393, 20410, 20512, 20537, 20538, 20540, 20823, 20923, 20943, 20987, 21032, 21202, 21245, 21262, 21440, 21632, 21668, 21688, 21696, 21766, 21986, 22123, 22157, 22259, 22300, 22301, 22357, 22453, 22474, 22490, 22547, 22702, 22707, 22775, 22776, 22789, 22844, 22872, 23049, 23184, 23409, 23482, 23630, 23727, 23728, 23739, 23928, 24127, 24243, 24286, 24463, 24474, 24485, 24489, 24497, 24533, 24573, 24760, 24775, 25087, 25135, 25168, 25179, 25187, 25193, 25194, 25252, 25311, 25312, 25406, 25435, 25469, 25535, 25593, 25606, 25610, 25625, 25650, 25798, 25804, 25810, 25814, 25847, 25855, 25896, 25918, 26096, 26176, 26180, 26202, 26206, 26211, 26235, 26254, 26331, 26395, 26431, 26541, 26563, 26654, 26671, 26808, 26859, 26963, 27109, 27133, 27258]

In [13]:
# extend with nans 
failed.extend(links[links.isnull().any(axis=1)].index)

## Imdb parsing

In [18]:
# ! pip install imdbpy
from imdb import IMDb
ia = IMDb()
imdb_movies = {}
imdb_failed = [118114]

In [21]:
for idx in tqdm(failed):
    try:
        imdb_movies[links['movieId'][idx]] = ia.get_movie(links['imdbId'][idx])
    except:
        imdb_failed.append(idx)

2019-04-05 09:27:00,521 CRITICAL [imdbpy] /home/dev/anaconda3/lib/python3.7/site-packages/imdb/_exceptions.py:34: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'http://www.imdb.com/title/tt0118114/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 404: 'Not Found'>},); kwds: {}
Traceback (most recent call last):
  File "/home/dev/anaconda3/lib/python3.7/site-packages/imdb/parser/http/__init__.py", line 231, in retrieve_unicode
    response = uopener.open(url)
  File "/home/dev/anaconda3/lib/python3.7/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/home/dev/anaconda3/lib/python3.7/urllib/request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "/home/dev/anaconda3/lib/python3.7/urllib/request.py", line 563, in error
    result = self._call_chain(*args)
  File "/home/dev/anaconda3/lib/python3.7/urllib/request.py", line 503, in _call_chain
 

In [58]:
import pickle
pickle.dump(imdb_movies,  open( "data/infos_imdb.p", "wb" ))

# Infos Processing

In [216]:
tmdb_info = pickle.load(open( "data/infos_tmdb.p", "rb" ))
imdb_info = pickle.load(open( "data/infos_imdb.p", "rb" ))

### TMDB

In [217]:
# tmdb_info[1]
# adult
# collection: family of films
# budget
# genres
# original_language
# overview
# popularity
# production companies: set of ids
# production_countries: set of labels
# release_date
# revenue
# runtime
# spoken_languages
# tagline
# title
# vote_average
# vote_count
movie_infos = {}
for i in tmdb_info.keys():
    movie = tmdb_info[i]
    raw = {}
    raw['adult'] = movie['adult']
    raw['collection'] = None
    if movie['belongs_to_collection'] is not None:
        raw['collection'] =  movie['belongs_to_collection']['id']
    raw['genres'] = [i['id'] for i in movie['genres']] 
    raw['original_language'] = movie['original_language']
    raw['overview'] = movie['overview']
    raw['popularity'] = movie['popularity']
    raw['production_companies'] = [i['id'] for i in movie['production_companies']]
    raw['production_countries'] = [i['iso_3166_1'].lower() for i in movie['production_countries']]
    if movie['release_date']:
        raw['release_year'] = int(movie['release_date'].split('-')[0])
        raw['release_month'] = int(movie['release_date'].split('-')[1])
    else:
        raw['release_year'] = 0
        raw['release_month'] = 0
    raw['revenue'] = movie['revenue']
    raw['runtime'] = movie['runtime']
    raw['spoken_languages'] = [i['iso_639_1'] for i in movie['spoken_languages']]
    raw['tagline'] = movie['tagline']
    raw['title'] = movie['title']
    raw['vote_average'] = movie['vote_average']
    raw['vote_count'] = movie['vote_count']
    movie_infos[i] = raw

In [218]:
tmdb_info[1]

{'adult': False,
 'backdrop_path': '/dji4Fm0gCDVb9DQQMRvAI8YNnTz.jpg',
 'belongs_to_collection': {'id': 10194,
  'name': 'Toy Story Collection',
  'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',
  'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'},
 'budget': 30000000,
 'genres': [{'id': 16, 'name': 'Animation'},
  {'id': 35, 'name': 'Comedy'},
  {'id': 10751, 'name': 'Family'}],
 'homepage': 'http://toystory.disney.com/toy-story',
 'id': 862,
 'imdb_id': 'tt0114709',
 'original_language': 'en',
 'original_title': 'Toy Story',
 'overview': "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
 'popularity': 25.398,
 'poster_path': '/rhIRbceoE9lR4veEXuwCC2wARtG.jpg',
 'production_companies': [{'id': 3,
   'logo_path': '/1TjvGVDMYsj6J

### IMDB

In [219]:
tmdb_genres_dict = {}
tmdb_companies_dict = {}
for i in tmdb_info.keys():
    movie = tmdb_info[i]
    genres = [{'id': i['id'], 'name': i['name']} for i in movie['genres']] 
    companies = [{'id': i['id'], 'name': i['name']} for i in movie['production_companies']] 
    for i in genres:
        tmdb_genres_dict[i['name']] = i['id']
    for i in companies:
        tmdb_companies_dict[i['name']] = i['id']

In [220]:
tmdb_genres_dict['Sport'] = 1
tmdb_genres_dict['Sci-Fi'] = 878
tmdb_genres_dict['Biography'] = 99
tmdb_genres_dict['Musical'] = 10402
tmdb_genres_dict['Short'] = 2
tmdb_genres_dict['News'] = 3
tmdb_genres_dict['Film-Noir'] = 4

In [221]:
# imdb_movies[4207].data
# 1 (tmdb format) : imdb_movies[id].data['2'] (imdb format)
# adult: certificates
# budget: None
# genres: genres
# original_language: language codes
# overview: plot outline + plot
# popularity: None?
# production_companies: None
# production_countries: countries
# release_date: original air date
# revenue: None
# runtime: runtimes[0]
# spoken_languages: language codes
# tagline: None
# vote_average: rating
# vote_count: votes
# title: title
import calendar
month_to_num = dict((v,k) for k,v in enumerate(calendar.month_abbr))

for i in tqdm(imdb_movies.keys()):
    movie = imdb_movies[i].data
    raw = {}
    raw['adult'] = False
    raw['collection'] = None
    raw['genres'] = []
    if 'genres' in movie:
        raw['genres'] = [tmdb_genres_dict[i] for i in movie['genres']] 
    
    raw['original_language'] = ''
    if 'language codes' in movie:
        raw['original_language'] = movie['language codes'][0]
    try:
        raw['overview'] = movie['plot outline'] + ' '.join([i.split("::")[0] for i in movie['plot']])
    except: 
        try:
            raw['overview'] = ' '.join([i.split("::")[0] for i in movie['plot']])
        except:
            raw['overview'] = ''
    raw['popularity'] = 0
    # raw['production_companies'] = [tmdb_companies_dict[i['name']] for i in movie['production companies']]
    production_companies = []
    if 'production companies' in movie:
        for c in movie['production companies']:
            try:
                production_companies.append(tmdb_companies_dict[c['name']])
            except:
                pass
    raw['production_companies'] = production_companies
    raw['production_countries'] = []
    if 'country codes' in movie:
        raw['production_countries'] = [i.lower() for i in movie['country codes']]
    
    raw['release_year'] = 0
    if 'year' in movie:
        raw['release_year'] = int(movie['year'])
    try:
        raw['release_month'] = month_to_num[movie['original air date'].split()[1]]
    except:
        raw['release_month'] = 1
    raw['revenue'] = 0
    try:
        raw['runtime'] = int(movie['runtimes'][0])
    except:
        raw['runtime'] = 0
    try:
        raw['spoken_languages'] = movie['language codes']
    except:
        raw['spoken_languages'] = []
    raw['tagline'] = ''
    if 'title' in movie:
        raw['title'] = movie['title']
    else:
        raw['title'] = movies[movies['movieId'] == i]['title'].values[0]
    raw['vote_average'] = 0
    if 'rating' in movie:
        raw['vote_average'] = movie['rating']
    raw['vote_count'] = 0
    if 'votes' in movie:
        raw['vote_count'] = movie['votes']
    movie_infos[i] = raw

In [227]:
movie_infos[1]

{'adult': False,
 'collection': 10194,
 'genres': [16, 35, 10751],
 'original_language': 'en',
 'overview': "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
 'popularity': 25.398,
 'production_companies': [3],
 'production_countries': ['us'],
 'release_year': 1995,
 'release_month': 10,
 'revenue': 373554033,
 'runtime': 81,
 'spoken_languages': ['en'],
 'tagline': '',
 'title': 'Toy Story',
 'vote_average': 7.9,
 'vote_count': 9787}

In [225]:
# make sure we are not missing any movies
assert set(movie_infos.keys()) == (set(movies['movieId'].values))

In [226]:
pickle.dump(movie_infos,  open( "data/infos.p", "wb" ))

In [ ]:
movie_infos_int_keys = {}
for k in movie_infos.keys():
    movie_infos_int_keys[int(k)] = movie_infos[k]

In [ ]:
import json
with open('data/infos.json', 'w') as outfile:  
    json.dump(movie_infos_int_keys, outfile)